In [1]:
import time
start = time.time()

import os, sys
import json
import pandas as pd

#Data-processing Functions
from modules.processDemographics import *
from modules.processGeoData import *

#Global Variables
city = "bangalore"
targetPopulation = 100000 #10000000
averageStudents= 300 #1700 #300
averageWorkforce=20

## Data processing

In [2]:
if not os.path.exists("data/"+city):
    os.mkdir("data/"+city)   

#TODO (SS):for the city get the base data (replace this with csv once Preetam gives data)
print("processing data..")

cityGeojson = "data/base/"+city+"/city.geojson"
blrDF = parse_geospatial_data(cityGeojson)


cityProfile = "data/base/"+city+"/cityProfile.json"
ageDistribution, householdDistribution, schoolDistribution, demographics = process_city_profile(blrDF, cityProfile)

demographics = project_population(demographics, targetPopulation) #scale population in city to the target population
totalWorkingPopulation = (float(demographics[['age 20-24']].sum())*0.6)+\
                                                float(demographics[['age 30-34']].sum())+\
                                                float(demographics[['age 35-39']].sum())+\
                                                float(demographics[['age 40-44']].sum())+\
                                                float(demographics[['age 45-49']].sum())+\
                                                float(demographics[['age 50-54']].sum())+\
                                                float(demographics[['age 55-59']].sum())

workplaceNeeded = totalWorkingPopulation/ averageWorkforce

print("getting parameters ready ....") #create DataFrames
totalNumberOfWards = len(demographics['wardNo'].values)
averageHouseholds = 4 #cityDemographicsDF['Mean Household Size'].describe()['mean']
commonArea = commonAreaLocation(blrDF)
print("creating and populating virtual city..")

processing data..
getting parameters ready ....
creating and populating virtual city..


In [3]:
# templateJSON={
#     "city": "banglore",
#     "age":{
#         "bins": ["0-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49", "50-54", "55-59", "60-64", "65-69", "70-74", "75-79", "80+"],
#         "weights": [0.073, 0.078, 0.083, 0.086, 0.102, 0.11, 0.098, 0.081, 0.071, 0.059, 0.049, 0.037, 0.03, 0.021, 0.01, 0.007, 0.006]

#     },
#     "householdSize":{
#         "bins": [1,2,3,4,5,6],
#         "weights": [0.0417, 0.1308, 0.2228, 0.3077, 0.1530, 0.1439]
#     },
#     "schoolsSize":{
#         "bins": [1,2,3,4,5,6], 
#         "weights": [0.0417, 0.1308, 0.2228, 0.3077, 0.1530, 0.1439]
#     },   
#     "demographics": json.loads(demographics[['Ward Name', 'Ward No.', 'Area (in sq km)',
#        'Total Population (in thousands)', 'Total no. of Households']].to_json(orient="records"))
# }

# with open("templateDataStore.json", "w") as f:
#     f.write(json.dumps(templateJSON))


In [4]:
time.time() - start

6.217562437057495

## instantiating individuals and households

In [8]:
start = time.time()

In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
__name__ = "Module to assign individuals to houses"
___author__ = "Sarath"


def assign_individuals_to_houses(targetPopulation, wards, totalHousehold, ageDistribution=[0.073, 0.078, 0.083, 0.086, 0.102, 0.11, 0.098, 0.081, 0.071, 0.059, 0.049, 0.037, 0.03, 0.021, 0.01, 0.007, 0.005], householdDistribution=[0.0417, 0.1308, 0.2228, 0.3077, 0.1530, 0.0726, 0.0645, 0.0054, 0.0015]):
    N = targetPopulation
    dictlist_individuals = [dict() for x in range(N)]

    H = int(targetPopulation/totalHousehold)
    dictlist_houses = [dict() for x in range(H)]

    W = wards

    # age distribution
    age_values = np.arange(0,81,1)
    age_distribution_over_gap5 = ageDistribution

    age_distribution = []
    for i in range(0,len(age_distribution_over_gap5)-1):
        for j in range(0,5):
            age_distribution.append(age_distribution_over_gap5[i]/5)
    age_distribution.append(age_distribution_over_gap5[16])
    age_distribution = np.array(age_distribution)
    age_distribution = age_distribution/sum(age_distribution)

    # household size distributin
    household_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    household_dist = householdDistribution  #1,2,3,4,5,6,7-10,11-14,15+
    household_distribution = household_dist[0:6]
    for i in range(0,4):
        household_distribution.append(household_dist[6]/4)
    for i in range(0,4):
        household_distribution.append(household_dist[7]/4)
    household_distribution.append(household_dist[8])
    household_distribution = np.array(household_distribution)/np.sum(household_distribution)
    mean_household_size = np.matmul(household_sizes, household_distribution)    
    
    # create individuals with desired age distribution
    individuals = {'id': np.arange(0,N), 'age':np.random.choice(age_values,N,p=age_distribution), 'household':np.ones((1,N))[0]*-1}
    individuals = pd.DataFrame(individuals) #after households DF is ready - add lat, lon, ward no to individuals || call functions in workplace_assignment.py

    # create households with desired household-size distribution
    #Ward No, Lat and Lon - would be city data (geojson of city)
    households = {'id':np.arange(0,H), 'Ward No': np.random.choice(np.arange(0,W),H), 'Lat':np.arange(0,H), 'Long':np.arange(0,H), 'people staying':np.random.choice(household_sizes, H, p=household_distribution), 'individuals':[[] for x in range(0,H)],  'flag':[0 for x in range(0,H)]}
    households = pd.DataFrame(households)


    # home-people assignment
    already_assigned = []

    for i in range(0,H):
        #print(i)
        if len(already_assigned)<N-6:
            if households['people staying'].values[i] == 1:
                possible_list = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=22,individuals['age'].values<=30)),already_assigned)]
                if (len(possible_list)>0):
                    choose_index = np.random.choice(len(possible_list))
                    already_assigned.append(int(possible_list.iloc[choose_index]['id']))
                    individuals.at[int(possible_list.iloc[choose_index]['id']),'household']=int(i)
                    households.at[i,'individuals'].append(int(possible_list.iloc[choose_index]['id']))
                    households.at[i,'flag']=1
            elif households['people staying'].values[i] == 2:
                possible_list = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=22,individuals['age'].values<=30)),already_assigned)]
                if (len(possible_list)>=2):
                    choose_index = np.random.choice(len(possible_list),2)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
            elif households['people staying'].values[i] == 3:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=22,individuals['age'].values<=40)),already_assigned)]
                if np.random.choice(1)==1:
                    possible_list2 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=10),already_assigned)]
                else:
                    possible_list2 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=60),already_assigned)]

                if (len(possible_list1)>=2) and (len(possible_list2)>=1):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1

                    choose_index = np.random.choice(len(possible_list2))
                    already_assigned.append(int(possible_list2.iloc[choose_index]['id']))
                    individuals.at[int(possible_list2.iloc[choose_index]['id']),'household']=int(i)
                    households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index]['id']))                    
                    households.at[i,'flag']=1
                    
            elif households['people staying'].values[i] == 4:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=30,individuals['age'].values<=55)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=5,individuals['age'].values<=30)),already_assigned)]

                if (len(possible_list1)>=2) and (len(possible_list2)>=2):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1

                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
            elif households['people staying'].values[i] == 5:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=40,individuals['age'].values<=55)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=5,individuals['age'].values<=21)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=22,individuals['age'].values<=30)),already_assigned)]
                if (len(possible_list1)>=2) and (len(possible_list2)>=2) and (len(possible_list3)>=1):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1

                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1

                    choose_index = np.random.choice(len(possible_list3))
                    already_assigned.append(int(possible_list3.iloc[choose_index]['id']))
                    individuals.at[int(possible_list3.iloc[choose_index]['id']),'household']=int(i)
                    households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index]['id']))
                    households.at[i,'flag']=1
            elif households['people staying'].values[i] == 6:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                if (len(possible_list1)>=2) and (len(possible_list2)>=2) and (len(possible_list3)>=2):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
             
            elif households['people staying'].values[i] == 7:
                
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=2) and (len(possible_list2)>=2) and (len(possible_list3)>=2) and (len(possible_list4)>=1):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),1, replace=False)
                    already_assigned.append(int(possible_list4.iloc[choose_index]['id']))
                    individuals.at[int(possible_list4.iloc[choose_index]['id']),'household']=int(i)
                    households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index]['id']))
                    households.at[i,'flag']=1
    	     
            elif households['people staying'].values[i] == 8:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=2) and (len(possible_list2)>=2) and (len(possible_list3)>=2) and (len(possible_list4)>=2):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    	               
            elif households['people staying'].values[i] == 9:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=2) and (len(possible_list2)>=2) and (len(possible_list3)>=2) and (len(possible_list4)>=3):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),3, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    	                                   
            elif households['people staying'].values[i] == 10:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=2) and (len(possible_list2)>=2) and (len(possible_list3)>=2) and (len(possible_list4)>=4):
                    choose_index = np.random.choice(len(possible_list1),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    	                                   
            elif households['people staying'].values[i] == 11:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=3) and (len(possible_list2)>=2) and (len(possible_list3)>=2) and (len(possible_list4)>=4):
                    choose_index = np.random.choice(len(possible_list1),3, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    	                                   
                        
            elif households['people staying'].values[i] == 12:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=4) and (len(possible_list2)>=2) and (len(possible_list3)>=2) and (len(possible_list4)>=4):
                    choose_index = np.random.choice(len(possible_list1),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    	                                   
            elif households['people staying'].values[i] == 13:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=4) and (len(possible_list2)>=2) and (len(possible_list3)>=3) and (len(possible_list4)>=4):
                    choose_index = np.random.choice(len(possible_list1),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),3, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
            elif households['people staying'].values[i] == 14:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=5) and (len(possible_list2)>=2) and (len(possible_list3)>=3) and (len(possible_list4)>=4):
                    choose_index = np.random.choice(len(possible_list1),5, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),3, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
            elif households['people staying'].values[i] == 15:
                possible_list1 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=21,individuals['age'].values<=30)),already_assigned)]
                possible_list2 = individuals.iloc[np.setdiff1d(np.where(np.logical_and(individuals['age'].values>=45,individuals['age'].values<=55)),already_assigned)]
                possible_list3 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values>=65),already_assigned)]
                possible_list4 = individuals.iloc[np.setdiff1d(np.where(individuals['age'].values<=20),already_assigned)]
                if (len(possible_list1)>=6) and (len(possible_list2)>=2) and (len(possible_list3)>=3) and (len(possible_list4)>=4):
                    choose_index = np.random.choice(len(possible_list1),6, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list1.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list1.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list1.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list2),2, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list2.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list2.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list2.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
    
                    choose_index = np.random.choice(len(possible_list3),3, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list3.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list3.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list3.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1
                        
                    choose_index = np.random.choice(len(possible_list4),4, replace=False)
                    for j in range(0,len(choose_index)):
                        already_assigned.append(int(possible_list4.iloc[choose_index[j]]['id']))
                        individuals.at[int(possible_list4.iloc[choose_index[j]]['id']),'household']=int(i)
                        households.at[i,'individuals'].append(int(possible_list4.iloc[choose_index[j]]['id']))
                        households.at[i,'flag']=1                                   
                        
    # assign unassigned individuals to households

    unassigned_individuals_ids =  individuals.loc[individuals['household']==-1]['id'].values
    unassigned_households_ids = households.loc[households['flag']==0]['id'].values
    # first look for any unoccupied household
    for i in range(0,len(unassigned_households_ids)):
        #print(i)
        #print(len(unassigned_individuals_ids)>=households.loc[i]['people staying'])
        if len(unassigned_individuals_ids)>=households.loc[unassigned_households_ids[i],'people staying']:
            indices = np.random.choice(len(unassigned_individuals_ids), households.loc[unassigned_households_ids[i]]['people staying'], replace=False)
            ind_ids = unassigned_individuals_ids[indices]
            households.at[unassigned_households_ids[i],'flag']=1
            for j in range(0,len(ind_ids)):
                households.at[unassigned_households_ids[i],'individuals'].append(ind_ids[j])
                individuals.at[ind_ids[j],'household'] = int(unassigned_households_ids[i])
                
            #print(len(unassigned_individuals_ids))
            unassigned_individuals_ids = np.setdiff1d(unassigned_individuals_ids,ind_ids)

    unassigned_households_ids = households.loc[households['flag']==0]['id'].values

    # if any more unassigned individuals, randomly assign them to houses and increase the household size

    if len(unassigned_individuals_ids)>0:
        if len(unassigned_households_ids)>0:
            for j in range(0,len(unassigned_individuals_ids)):
                house_index = np.random.choice(len(unassigned_households_ids))
                individuals.at[unassigned_individuals_ids[j],'household'] = unassigned_households_ids[house_index]
                #households.at[unassigned_households_ids[house_index],'people staying'] = households.loc[house_index,'people staying'] + 1
                households.at[unassigned_households_ids[house_index],'flag'] = 1
                households.at[unassigned_households_ids[house_index], 'individuals'].append(unassigned_individuals_ids[j])
        else:
            house_indices =  np.random.choice(len(households),len(unassigned_individuals_ids), replace=False)
            for j in range(0,len(unassigned_individuals_ids)):
                households.at[house_indices[j],'people staying'] = households.loc[house_indices[j],'people staying']+1
                households.at[house_indices[j],'individuals'].append(unassigned_individuals_ids[j])
                individuals.at[unassigned_individuals_ids[j],'household']=house_indices[j]
                households.at[house_indices[j],'flag'] = 1
                households.at[house_indices[j], 'individuals'].append(unassigned_individuals_ids[j])

    unassigned_households_ids = households.loc[households['flag']==0]['id'].values
    households = households.loc[households['flag']!=0]
    individuals['household']=individuals['household'].astype(int)
    return individuals, households

individuals, households = assign_individuals_to_houses(targetPopulation, totalNumberOfWards, averageHouseholds, ageDistribution, householdDistribution)

In [11]:
import numpy as np

houseNumbers = individuals['household'].values
#ward assignments based on ID column
wardBounds = demographics.copy()
wardBounds = wardBounds.rename(columns={"wardNo": "Ward No"})
wardBounds = wardBounds[['Ward No', 'wardBounds']] #sorted by ward numbers
households = pd.merge(households, wardBounds, on=['Ward No'])

households['location'] = households.apply(lambda row: (np.random.choice([row['wardBounds'][0],row['wardBounds'][2]]), np.random.choice([row['wardBounds'][1], row['wardBounds'][3]])), axis=1)
households['lat'] = households.apply(lambda row: row['location'][1], axis=1)
households['lon'] = households.apply(lambda row: row['location'][0], axis=1)
households['household'] = households['id'].values
individuals = pd.merge(individuals, households[['household', 'Ward No', 'lat', 'lon']], on='household')
individuals = individuals.rename(columns={"Ward No": "ward"})

In [10]:
time.time()-start

911.9199621677399

## Instantiate schools in each ward

In [ ]:
start = time.time()

## Instantiate Community Centre for Each Ward

In [6]:
cc = pd.DataFrame()
cc['wardNo'] = blrDF['wardNo'].values
cc['location'] = blrDF['wardCentre'].values

In [12]:
schools = pd.DataFrame()
averageStudents = 300
wards = demographics['wardNo'].values
bounds = demographics['wardBounds'].values
totalSchools = demographics.apply(lambda row: int(((row['age 0-4']*0.1) +\
                                                    row['age 5-9'] + \
                                                    row['age 10-14'] + \
                                                    row['age 15-19'] + \
                                                    (row['age 20-24']*0.3))/ averageStudents), axis=1) 

ward = []
location = []

for i in range(len(wards)):
    lon1 = bounds[i][0]
    lat1 = bounds[i][1]
    lon2 = bounds[i][2]
    lat2 = bounds[i][3]

    for house in range(totalSchools[i]):
        ward.append(i+1)
        location.append((uniform(lon1, lon2), uniform(lat1, lat2)))

schools['ward'] = ward
schools['location'] = location
schools = schools.reset_index()
schools = schools.rename(columns={"index":"ID"})

## Adding Workplaces - Offices and Community Space on GeoMap

Number of Community Space = 1/ward 

In [13]:
averageWorking = 20
ward = []
location = []
workplaces = pd.DataFrame()

wards = blrDF['wardNo'].values.tolist()
bounds = blrDF['wardBounds'].values.tolist()

for space in range(int(workplaceNeeded)):

    boundIndex = int(uniform(0, len(bounds)))

    lon1 = bounds[boundIndex][0]
    lat1 = bounds[boundIndex][1]
    lon2 = bounds[boundIndex][2]
    lat2 = bounds[boundIndex][3]

    ward.append(wards[boundIndex])
    location.append((uniform(lon1, lon2), uniform(lat1, lat2)))



workplaces['ward'] = ward
workplaces['location'] = location
workplaces = workplaces.reset_index()
workplaces = workplaces.rename(columns={"index":"ID"})

#Combining the IDs for schools and workplaces
schoolID = schools['ID'].values
workplaceID = [schoolID[-1]+1 + index for index in workplaces['ID'].values]
workplaces['ID'] = workplaceID

In [ ]:
time.time() - start

In [14]:
start = time.time()
import scipy.stats as stats
import numpy as np
import pandas as pd 

In [15]:
wards = demographics[['wardNo', 'wardIndex', 'neighbors']]

In [16]:

# generate s sample of size of workplace 
#def gen_wp_size(count=1, a=3.26, c=0.97, m_max=2870):
    #function to generate a truncated Zipf sample

#    vals = np.arange(m_max)
#    p_nplus = np.arange(float(m_max))
#    for m in range(m_max):
#        p_nplus[m] =  ((( (1+m_max/a)/(1+m/a))**c) -1) / (((1+m_max/a)**c) -1)

#    p_nminus = 1.0 - p_nplus
#    p_n = np.arange(float(m_max))
#    prev=0.0
#    for m in range(1, m_max):
#        p_n[m] = p_nminus[m] - prev
#        prev = p_nminus[m] 

#    bzipf = stats.rv_discrete (name='bzipf', values=(vals, p_n))
#    rval = bzipf.rvs(size=count)
    #print(rval)
#    return rval


# findout neighbours of a given ward
def neighbouring_wards_ids(input_ward):
    return np.array(str.split(wards.loc[wards['wardNo']==input_ward,'neighbors'].values[0],','),dtype=int)
    
# compute Euclidean distance    
def distance(lat1, long1, lat2, long2):
    return np.sqrt((lat1 - lat2)**2 + (long1-long2)**2)
    
# findout possible wokrplaces for an individual by looking at nearby wards
def possible_workplaces_ids(input_ward):
    neighbouring_wards = neighbouring_wards_ids(input_ward)
    temp = []
    for j in range(0,len(neighbouring_wards)):
        temp = np.concatenate((temp,np.array(workplaces.loc[workplaces['ward']==neighbouring_wards[j]]['ID'].values) ))
    return temp
    

def assign_schools_and_workplaces(workplaces, schools, individuals):
    # generate workplaces size distribution
    count=1
    a=3.26
    c=0.97
    m_max=2870
    
    vals = np.arange(m_max)
    p_nplus = np.arange(float(m_max))
    for m in range(m_max):
        p_nplus[m] =  ((( (1+m_max/a)/(1+m/a))**c) -1) / (((1+m_max/a)**c) -1)

    p_nminus = 1.0 - p_nplus
    p_n = np.arange(float(m_max))
    prev=0.0
    for m in range(1, m_max):
        p_n[m] = p_nminus[m] - prev
        prev = p_nminus[m] 

    bzipf = stats.rv_discrete (name='bzipf', values=(vals, p_n))
    
    # generate schools size distribution
    schoolsize_values = np.arange(50,901,1)
    schoolsize_distribution_over_gap100 =[ 0.0184, 0.1204, 0.2315, 0.2315, 0.1574, 0.0889, 0.0630, 0.0481, 0.0278, 0.0130]
    # 50-99, 100-199, ..., 800 - 899, 900+
    schoolsize_distribution = []
    for i in range(1,len(schoolsize_distribution_over_gap100)-1):
        for j in range(0,100):
            schoolsize_distribution.append(schoolsize_distribution_over_gap100[i]/100)

    for i in range(0,50):
        schoolsize_distribution.insert(0,schoolsize_distribution_over_gap100[0]/50)

    schoolsize_distribution.append(schoolsize_distribution_over_gap100[len(schoolsize_distribution_over_gap100)-1])
    schoolsize_distribution = np.array(schoolsize_distribution)
    schoolsize_distribution = schoolsize_distribution/np.sum(schoolsize_distribution)
    mean_schoolsize = np.matmul(schoolsize_values, schoolsize_distribution)


#     wards = wardNeighbors
    W = len(wards)

    WP = len(workplaces)
    # insert some columns for further processing
    workplaces.insert(2,"workforce", [0 for x in range(0,WP)]) 
    workplaces.insert(3,"workers", [[] for x in range(0,WP)])

    S = len(schools)
    schools.insert(2,"students", [[] for x in range(0,S)])
    schools.insert(3,"strength", [0 for x in range(0,S)])
    schools.insert(4,"capacity", np.random.choice(schoolsize_values, S, p=schoolsize_distribution))

    workDistance = []

    # generate capacity according to workspace size distribution
    capacity = []
    for i in range(0,WP):
        capacity.append(bzipf.rvs(size=1)[0])

    workplaces.insert(4,"capacity", capacity)

    # keep track of individuals already assigned
    already_assigned = []
    count = 0
    workforce = []
    studentforce = []
    already_assigned_students = []
    # assign individuals
    for i in range(0,len(individuals)):
        # print(i/len(individuals))
        # individuals to workplaces
        if individuals.loc[i,'age']>=22 and individuals.loc[i,'age']<=55: 
            workforce.append(i)
            count = count+1
            lat = individuals.loc[i,'lat']
            long = individuals.loc[i,'lon']
            possible_workplace_ids =  possible_workplaces_ids(individuals.loc[i,'ward'])
            distances = []
            for j in range(0,len(possible_workplace_ids)):
                distances.append(distance(lat,long,workplaces.loc[workplaces['ID']==int(possible_workplace_ids[j]),'location'].values[0][1],workplaces.loc[workplaces['ID']==int(possible_workplace_ids[j]),'location'].values[0][0]))
            distances = np.array(distances)
            distances = distances/sum(distances)
            add_to_workplace_id = int(possible_workplace_ids[np.random.choice(len(possible_workplace_ids),p=distances)])
            if workplaces.loc[workplaces.loc[workplaces['ID']==int(add_to_workplace_id),'ID'].index[0], 'workforce'] < workplaces.loc[workplaces.loc[workplaces['ID']==int(add_to_workplace_id),'ID'].index[0], 'capacity']:
                individuals.at[i,'workplace'] = add_to_workplace_id
                workplaces.at[workplaces.loc[workplaces['ID']==int(add_to_workplace_id),'ID'].index[0],'workforce'] = workplaces.loc[workplaces.loc[workplaces['ID']==int(add_to_workplace_id),'ID'].index[0],'workforce']+1
                workplaces.at[workplaces.loc[workplaces['ID']==int(add_to_workplace_id),'ID'].index[0],'workers'].append(i)
                already_assigned.append(i)
        # individuals to schools 
        elif individuals.loc[i,'age']<=21 and individuals.loc[i,'age']>=5:
            studentforce.append(i)
            lat = individuals.loc[i,'lat']
            long = individuals.loc[i,'lon']
            possible_school_id1 = schools.loc[schools['ward']==individuals.loc[i,'ward']]['ID'].values
            possible_school_id = []
            if len(possible_school_id1) > 0:
                for j in range(0,len(possible_school_id1)):
                    if schools.loc[schools.loc[schools['ID']==possible_school_id1[j],'ID'].index[0],'capacity'] > schools.loc[schools.loc[schools['ID']==possible_school_id1[j],'ID'].index[0],'strength']:
                        possible_school_id.append(possible_school_id1[j])
                possible_school_id = np.array(possible_school_id)
            if len(possible_school_id) > 0:
                index = np.random.choice(len(possible_school_id))
                individuals.at[i,'school'] = possible_school_id[index]
                schools.at[schools.loc[schools['ID']==possible_school_id[index]].index[0],'students'].append(i)
                schools.at[schools.loc[schools['ID']==possible_school_id[index]].index[0],'strength'] = schools.loc[schools.loc[schools['ID']==possible_school_id[index]].index[0],'strength'] +1 
                already_assigned_students.append(i)
                
    # randomly assign unassigned individuals to workplaces and schools
    # first check for workplaces that are not full
    for i in range(0,len(workplaces)):
        if workplaces.loc[i,'capacity'] > workplaces.loc[i,'workforce']:
            d = workplaces.loc[i,'capacity'] - workplaces.loc[i,'workforce']
            if len(np.setdiff1d(workforce, already_assigned)) >=d:
                add_to_workplace_id = np.random.choice(np.setdiff1d(workforce, already_assigned), d, replace=False)
                for j in range(0,d):
                    individuals.at[add_to_workplace_id[j],'workplace'] = workplaces.loc[i,'ID']
                    already_assigned.append(add_to_workplace_id[j])
                    workplaces.at[i,'workers'].append(add_to_workplace_id[j])
                    workplaces.at[i,'workforce'] = workplaces.loc[i,'workforce']+1
    # first check if schools are not full
    for i in range(0,len(schools)):
        if schools.loc[i,'capacity'] > schools.loc[i,'strength']:
            d = schools.loc[i,'capacity'] - schools.loc[i,'strength']
            if len(np.setdiff1d(studentforce, already_assigned_students)) >=d:
                add_to_school_id = np.random.choice(np.setdiff1d(studentforce, already_assigned_students), d, replace=False)
                for j in range(0,d):
                    individuals.at[add_to_school_id[j],'school'] = schools.loc[i,'ID']
                    already_assigned_students.append(add_to_school_id[j])
                    schools.at[i,'students'].append(add_to_school_id[j])
                    schools.at[i,'strength'] = schools.loc[i,'strength']+1
    # if anyone has unassigned workplace or school, assign him/her randomly
    for i in range(0,len(individuals)):
        if individuals.loc[i,'age']>=22 and individuals.loc[i,'age']<=55 and (not i in already_assigned):
            lat = individuals.loc[i,'lat']
            long = individuals.loc[i,'lon']
            add_to_workplace_id = np.random.choice(WP)
            individuals.at[i,'workplace'] = add_to_workplace_id
            workplaces.at[int(add_to_workplace_id), 'workforce'] =workplaces.loc[int(add_to_workplace_id), 'workforce']+1 
            workplaces.at[int(add_to_workplace_id),'workers'].append(i)

        elif individuals.loc[i,'age']<=21 and individuals.loc[i,'age']>=5 and (not i in already_assigned_students):
            lat = individuals.loc[i,'lat']
            long = individuals.loc[i,'lon']
            add_to_school_id = np.random.choice(S)
            individuals.at[i,'school'] = add_to_school_id
            schools.at[int(add_to_school_id), 'strength'] =schools.loc[int(add_to_school_id), 'strength']+1 
            schools.at[int(add_to_school_id),'students'].append(i)
    return workplaces, schools, individuals

In [17]:
workplaces, schools, individuals = assign_schools_and_workplaces(workplaces, schools, individuals)


In [18]:
individuals

,id,age,household,ward,lat,lon,workplace,school
0,0,44,10109,48,13.008525,77.613475,280.0,NaN
1,52983,18,10109,48,13.008525,77.613475,NaN,0.0
2,54641,25,10109,48,13.008525,77.613475,850.0,NaN
3,90117,34,10109,48,13.008525,77.613475,1627.0,NaN
4,1,55,18542,40,13.051762,77.467364,1992.0,NaN
...,...,...,...,...,...,...,...,...
99595,99535,24,14179,192,12.834012,77.582944,77.0,NaN
99596,99640,28,8286,29,13.012568,77.626149,166.0,NaN
99597,99673,23,3704,190,12.884789,77.629520,1895.0,NaN
99598,99779,43,24378,185,12.893236,77.559484,1423.0,NaN


In [19]:
time.time()-start

4430.532737255096

## Dump of static data as json

In [21]:
def getDistances(row, cc):
    findCommunityCentre = cc[int(row["ward"])]
    lat1 = row['lat']
    lon1 = row['lon']

    lat2 = findCommunityCentre[1]
    lon2 = findCommunityCentre[0]

    return distance(lat1, lon1, lat2, lon2)

def getWorkplaceType(row):
    if not np.isnan(row['workplace']) and np.isnan(row['school']):
        return 1
    if np.isnan(row['workplace']) and not np.isnan(row['school']):
        return 2
    if np.isnan(row['workplace']) and np.isnan(row['school']):
        return 0

individuals['CommunityCentreDistance'] = individuals.apply(getDistances, axis=1, args=(commonArea['location'].values,))
individuals['workplaceType']=individuals.apply(getWorkplaceType, axis=1)

In [30]:
start = time.time()

In [31]:
individuals.to_json("data/"+city+"/individuals.json", orient='records')
households[['id', 'Ward No', 'lat', 'lon']].to_json("data/"+city+"/houses.json", orient='records')
schools.to_json("data/"+city+"/schools.json", orient='records')
workplaces.to_json("data/"+city+"/workplaces.json", orient='records')
cc.to_json("data/"+city+"/commonArea.json", orient='records')
blrDF.to_json("data/"+city+"/map.geojson")
computeWardCentreDistance(blrDF, "data/"+city+"/wardCentreDistance.json")

/home/sharad/Work/CoviD-19/edaDev/lib/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [32]:
time.time()-start

1.0471553802490234